In [1]:
import time
import numpy as np
import tensorflow as tf
import matplotlib.pylab as plt
from sklearn.datasets import fetch_mldata

In [2]:
x_ = tf.placeholder(tf.float32,shape=(None,28,28,1))
y_ = tf.placeholder(tf.float32,shape=(None,10))

In [3]:
conv1_features = 20
max_pool_size1 = 2
#畳み込み層1
conv1_w = tf.Variable(tf.truncated_normal([5,5,1,conv1_features],stddev=0.1), dtype=tf.float32)
conv1_b = tf.Variable(tf.constant(0.1,shape=[conv1_features]),dtype=tf.float32)
conv1_c2 = tf.nn.conv2d(x_,conv1_w,strides=[1,1,1,1],padding="SAME")
conv1_relu = tf.nn.relu(conv1_c2+conv1_b)
conv1_mp = tf.nn.max_pool(conv1_relu, ksize=[1,max_pool_size1,max_pool_size1,1],strides=[1,max_pool_size1,max_pool_size1,1],padding="SAME")
#畳み込み層2
conv2_features=50
max_pool_size2 = 2
conv2_w = tf.Variable(tf.truncated_normal([5,5,conv1_features,conv2_features],stddev=0.1),dtype=tf.float32)
conv2_b = tf.Variable(tf.constant(0.1,shape=[conv2_features]),dtype=tf.float32)
conv2_c2 = tf.nn.conv2d(conv1_mp,conv2_w,strides=[1,1,1,1],padding="SAME")
conv2_relu = tf.nn.relu(conv2_c2+conv2_b)
conv2_mp = tf.nn.max_pool(conv2_relu,ksize=[1,max_pool_size2,max_pool_size2,1],strides=[1,max_pool_size2,max_pool_size2,1],padding="SAME")
#全結合層1
result_w = x_.shape[1] // (max_pool_size1*max_pool_size2)
result_h = x_.shape[2] // (max_pool_size1*max_pool_size2)
fc_input_size = result_w * result_h * conv2_features
fc_features = 500
s = conv2_mp.get_shape().as_list()
conv_result = tf.reshape(conv2_mp,[-1,s[1]*s[2]*s[3]])
fc1_w = tf.Variable(tf.truncated_normal([fc_input_size.value,fc_features],stddev=0.1),dtype=tf.float32)
fc1_b = tf.Variable(tf.constant(0.1, shape=[fc_features]),dtype=tf.float32)
fc1 = tf.nn.relu(tf.matmul(conv_result,fc1_w)+fc1_b)

fc2_w = tf.Variable(tf.truncated_normal([fc_features,10],stddev=0.1),dtype=tf.float32)
fc2_b = tf.Variable(tf.constant(0.1,shape=[10]),dtype=tf.float32)
y = tf.matmul(fc1,fc2_w)+fc2_b

In [4]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [16]:
EPOCH_NUM = 5
BATCH_SIZE = 1000
mnist = fetch_mldata("MNIST original",data_home=".")
mnist.data = mnist.data.astype(np.float32)
mnist.data /= 255

In [18]:
mnist.target = mnist.target.astype(np.int32)
N = 60000
train_x, test_x = np.split(mnist.data,[N])
train_y,test_y = np.split(mnist.target,[N])
train_x = train_x.reshape((len(train_x),28,28,1))
test_x = test_x.reshape((len(test_x),28,28,1))
train_y = np.eye(np.max(train_y)+1)[train_y]
test_y = np.eye(np.max(test_y)+1)[test_y]
print("Train")
with tf.Session() as sess:
    st = time.time()
    sess.run(tf.global_variables_initializer())
    for epoch in range(EPOCH_NUM):
        perm = np.random.permutation(N)
        total_loss = 0
        for i in range(0,N,BATCH_SIZE):
            batch_x = train_x[perm[i:i+BATCH_SIZE]]
            batch_y = train_y[perm[i:i+BATCH_SIZE]]
            total_loss += cross_entropy.eval(feed_dict={x_:batch_x,y_:batch_y})
            train_step.run(feed_dict={x_:test_x,y_:test_y})
        test_accuracy = accuracy.eval(feed_dict={x_: test_x, y_: test_y})
        if (epoch+1) % 1 == 0:
            ed = time.time()
            print("epoch:\t{}\ttotal loss:\t{}\tvaridation accuracy:\t{}\ttime:\t{}".format(epoch+1,total_loss,test_accuracy,ed-st))

Train


ResourceExhaustedError: OOM when allocating tensor with shape[10000,20,28,28]
	 [[Node: Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](_recv_Placeholder_0/_13, Variable/read)]]

Caused by op 'Conv2D', defined at:
  File "/usr/lib/python3.4/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.4/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.4/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.4/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.4/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, se